## Functions and Decorator Pattern

### Topics

- handle variable length arguments
- lambda expressions
- higher-order functions
- nested functions
- functions as returned values
- currying
- function decorators
- function and class decorator pattern

## Variable length arguments

- when not sure how many arguments will be passed to a function
- `*args` (non-keyworded variable length arguments)
- `**kwargs` (keyworded variable length arguments)
- e.g., built-in `print` function uses variable length arguments and keyworded arguments

### print(*object, sep=' ', end='\n', file=sys.stdout, flush=False)


In [22]:
# variable length arguments demo
def someFunction(a, b, c, *varargs, **kwargs):
    print('a = ', a)
    print('b = ', b)
    print('c = ', c)
    print('*args = ', varargs)
    print('type of args = ', type(varargs))
    print('**kwargs = ', kwargs)
    print('type of kwargs = ', type(kwargs))

In [23]:
# call someFunction with some arguments
someFunction(1, 'Apple', 4.5, 5, [2.5, 'b'], fname='Jake', num=1)

a =  1
b =  Apple
c =  4.5
*args =  (5, [2.5, 'b'])
type of args =  <class 'tuple'>
**kwargs =  {'fname': 'Jake', 'num': 1}
type of kwargs =  <class 'dict'>


## Lambda Functions/Expressions

- anonymous functions without names
- typically used in conjunction with higher order functions such as: `map()`, `reduce()`, `filter()`
- Reference: http://www.secnetix.de/olli/Python/lambda_functions.hawk

### lambda function properties and usage

- single line simple functions
- no explicit return keyword is used
- always contains an expression that is implictly returned
- can use a lambda definition anywere a function is expected without assigning to a variable
- syntax: 
```python
    lambda argument(s): expression
```

### difference between lambda and regular function

In [36]:
nums = input('Enter 5 numbers separated by space:')

Enter 5 numbers separated by space:2 9 8 100 76


In [37]:
nums

'2 9 8 100 76'

In [38]:
nums = list(map(int, nums.split()))

In [39]:
nums

[2, 9, 8, 100, 76]

In [24]:
# regular function
def func(x): return x**2

In [26]:
print(func(4))

16


In [25]:
type(func)

function

In [27]:
print(func)

<function func at 0x7ff27d01a170>


In [28]:
g = lambda x: x**2 # no name, no parenthesis, and no return keyword
# a function that takes x and returns x**2

In [31]:
g(2)

4

In [29]:
type(g)

function

In [30]:
print(g)

<function <lambda> at 0x7ff27d01a8c0>


## Higher-order functions

https://composingprograms.com/pages/16-higher-order-functions.html
- functions that manipulate other functions are called higher order functions
- functions take function(s) as argument(s)
    - typically, lambda expressions are passed as arguments
- functions can define and return a local function

In [32]:
# compute summations of n natural numbers
# func is a function applied to all the natural numbers between 1 and n inclusive
def sum_naturals(func, n):
    total, k = 0, 1
    while k <= n:
        total += func(k)
        k += 1
    return total

In [36]:
n = 100

In [37]:
# pass lambda function
print(f'sum of first {n} natural numbers = {sum_naturals(lambda x: x, n)}')

sum of first 100 natural numbers = 5050


In [38]:
# of course you can pass regular function
def even(n):
    return n if n%2 == 0 else 0

In [39]:
print(f'sum of even numbers from 1 to {n} = {sum_naturals(even, n)}')

sum of even numbers from 1 to 100 = 2550


In [40]:
# one-liner lambda
print(f'sum of odd numbers from 1 to {n} = {sum_naturals(lambda x: x if x%2==1 else 0, n)}')

sum of odd numbers from 1 to 100 = 2500


## Nested functions

- functions can be defined inside a function with local scope
- locally defined functions also have access to the names defined in their parent function
    - this technique is called lexical scoping
- helps keep the global frame clean and less cluter with functions that are only used inside some functions
- let's redefine sum_natural function again with local functions

In [41]:
# compute summations of n natural numbers
# by default find sum of all the natural numbers between 1 and n inclusive
def sum_naturals1(n, number_type="all"):
    def even(x):
        return x if x%2 == 0 else 0
    
    def odd(x):
        return x if x%2 == 1 else 0
    
    def func(x):
        # local function has access to global variables as well as parent frames
        if number_type == 'even':
            return x if x%2 == 0 else 0
        else:
            return x if x%2 == 1 else 0
            
    total, k = 0, 1
    while k <= n:
        if number_type != 'all':
            total += func(k)
        elif number_type == 'odd':
            total += odd(k)
        else:
            total += k
        k += 1
    return total

In [42]:
n = 100
print(f'sum of first {n} natural numbers = {sum_naturals1(n)}')

sum of first 100 natural numbers = 5050


In [43]:
print(f'sum of even numbers from 1 to {n} = {sum_naturals1(n, "even")}')

sum of even numbers from 1 to 100 = 2550


In [44]:
# sum of odd numbers from 1 to 100
print(f'sum of odd numbers from 1 to {n} = {sum_naturals1(n, "odd")}')

sum of odd numbers from 1 to 100 = 2500


## Functions as returned values

- functions can return a function
- locally defined functions maintain their parent environment when they are returned

In [45]:
def number_type(ntype='all'):
    def even(x):
        return x if x%2 == 0 else 0
    
    def odd(x):
        return x if x%2 == 1 else 0
    
    def _(x): # function to return x as it is; any()
        return x
    
    if ntype == 'all':
        return _
    elif ntype == 'even':
        return even
    else:
        return odd

In [46]:
n = 100
print(f'sum of first {n} natural numbers = {sum_naturals(number_type("all"), n)}')

sum of first 100 natural numbers = 5050


In [47]:
print(f'sum of even numbers from 1 to {n} = {sum_naturals(number_type("even"), n)}')

sum of even numbers from 1 to 100 = 2550


In [48]:
# sum of odd numbers from 1 to 100
print(f'sum of odd numbers from 1 to {n} = {sum_naturals(number_type("odd"), n)}')

sum of odd numbers from 1 to 100 = 2500


## Currying

- breaking down the evaluation of a function that takes multiple arguments into evaluating a sequence of single-argument functions
- also used in chaining multiple functions into one

- e.g., given a function `f(x, y)`, we can define a function `g(x)(y)` equivalent to `f(x, y)`
    - `g` is a higher-order function that takes in a single argument `x` and returns another function that takes in a single argument `y`
- a single function can be broken into multiple functions by chaining the smaller functions
    - `f(x) = h(g(x))`
    - the output of inner function `g(x)` is an input for outer function `h()`
    - this transformation is called **currying**

In [49]:
# single function that converts days to seconds
def daysToSeconds(days):
    return days * 24 * 60 * 60

In [50]:
# use the function
daysToSeconds(1)

86400

In [51]:
# currying application
# convert no. of days into seconds
def convertDaysToSeconds(f1, f2, f3):
    """Higer order function that takes 3 functions."""
    def f(x): 
        return f1(f2(f3(x)))
    return f 

def daysToHours(days): 
    """ Function converts days to hours."""
    return days * 24

def hoursToMinutes(hours): 
    """ Function converts hours to minutes."""
    return hours * 60

def minutesToSeconds(minutes): 
    """ Function converts minutes to seconds."""
    return  minutes * 60

In [55]:
# create a single function curry
days_to_seconds = convertDaysToSeconds(minutesToSeconds, hoursToMinutes, daysToHours)

In [56]:
days_to_seconds(1)

86400

In [57]:
# currying application; function with two arguments
# built-in pow function takes two arguments; 
# let's convert it into a function that takes a single argument
def curried_pow(x):
    def g(y):
        return pow(x, y)
    return g # function preserves the local variables, x and y

In [58]:
curried_pow(2)(3) # x=2, y=3; -> pow(2, 3)
# same as 2**3

8

In [59]:
# currying application 2
# function maps each element in alist with func() transformation
def my_map(alist, func):
    for i in range(len(alist)):
        alist[i] = func(alist[i])

In [60]:
# let's create a list of integers and map each to a different value
nums = list(range(1, 11))

In [61]:
# function maps each element in alist with func() transformation
def my_map(alist, func):
    for i in range(len(alist)):
        alist[i] = func(alist[i])

In [62]:
my_map(nums, curried_pow(2))

In [63]:
nums

[2, 4, 8, 16, 32, 64, 128, 256, 512, 1024]

## Built-in Function Decorators

- `functools` provides `wraps` and many built-in decorators that can be very useful
    - e.g., least recently used (LRU) caching
- See [https://docs.python.org/3/library/functools.html](https://docs.python.org/3/library/functools.html) for all the higher-order functions
- any callable object can be passed to a higher-order function to decorate it!
- See this tutorial for Callbe objects in Python - [https://realpython.com/python-callable-instances/](https://realpython.com/python-callable-instances/)
- built-in decorator example

In [1]:
from functools import cache

In [2]:
count = 0
@cache
def factorial(n):
    global count
    count += 1
    return n*factorial(n-1) if n else 1

In [3]:
factorial(100)

93326215443944152681699238856266700490715968264381621468592963895217599993229915608941463976156518286253697920827223758251185210916864000000000000000000000000

In [4]:
count

101

In [5]:
factorial(50) # use cachaed result

30414093201713378043612608166064768844377641568960512000000000000

In [6]:
count

101

In [7]:
factorial(102) # make 2 more recursive calls

961446671503512660926865558697259548455355905059659464369444714048531715130254590603314961882364451384985595980362059157503710042865532928000000000000000000000000

In [8]:
count

103

In [9]:
factorial.cache_info()

CacheInfo(hits=2, misses=103, maxsize=None, currsize=103)

In [10]:
# let's define our own int function using partial decorator
help(int)

Help on class int in module builtins:

class int(object)
 |  int([x]) -> integer
 |  int(x, base=10) -> integer
 |  
 |  Convert a number or string to an integer, or return 0 if no arguments
 |  are given.  If x is a number, return x.__int__().  For floating point
 |  numbers, this truncates towards zero.
 |  
 |  If x is not a number or if base is given, then x must be a string,
 |  bytes, or bytearray instance representing an integer literal in the
 |  given base.  The literal can be preceded by '+' or '-' and be surrounded
 |  by whitespace.  The base defaults to 10.  Valid bases are 0 and 2-36.
 |  Base 0 means to interpret the base from the string as an integer literal.
 |  >>> int('0b100', base=0)
 |  4
 |  
 |  Built-in subclasses:
 |      bool
 |  
 |  Methods defined here:
 |  
 |  __abs__(self, /)
 |      abs(self)
 |  
 |  __add__(self, value, /)
 |      Return self+value.
 |  
 |  __and__(self, value, /)
 |      Return self&value.
 |  
 |  __bool__(self, /)
 |      True if 

In [14]:
int(3.5)

3

In [15]:
int('11', base=2)

3

In [21]:
from functools import partial

basetwo = partial(int, base=2)
basetwo.__doc__ = 'Convert base 2 string to an int'

In [22]:
help(basetwo)

Help on partial in module functools:

functools.partial(<class 'int'>, base=2)
    Convert base 2 string to an int



In [23]:
basetwo('111111')

63

In [24]:
basetwo('1000')

8

In [25]:
assert basetwo('11') == 3

## Defining Function Decorators

- Python provides `wraps()` function to wrap any function that needs to be decorated
- https://realpython.com/primer-on-python-decorators/
- decorators are higher order functions
- decorators take another function as an argument and decorate it
- argument function is decorated by extending its behavior without explictly modifying the function itself

#### How about passing arguments to decorated function and its return values?
1. if the function being decorated takes arguments, provide arguments to the wrapper function
    - wrapper function wraps the argument function
2. if the function being decorated returns a value, call it with `return` statement
    - obviously, must be the last statement

### Real-world Applications

- we've already used: `@classmethod`, `@property`, `@staticmethod`
- many frameworks such as Flask, Django provide lots of decorators
    - e.g. @login_required; @app.route("/route_name"), etc.
    - simply define the functions using the library provided decorators!
- Python `functools` library provides many higher order decorators:
https://docs.python.org/3/library/functools.html

In [80]:
# a simple fruitless function without parameter
def hello(name=None):
    """Print Hello there!"""
    print("Hello there...")

In [76]:
hello()
# too simple... let's decorate hello

Hello there!


In [129]:
# a simple decorator function to decorate hello
def hello_decorator(func):
    """Decorate func."""
    
    def wrapper_hello_decorator(*args, **kwargs):
        """Wrapper for hello function."""
        # code ...
        # call the actual function
        ret = func(*args, **kwargs)
        # use ret value...
        if not args:
            print("stranger...")
        else:
            print(f"Great seeing you {args[0]} !!")
        print("Have a great time!")
        # code ...
    return wrapper_hello_decorator

In [103]:
# hello is decorated now, without modifying the original function
# just the behavior is modified by added extra print() before and after hello
hello_deco = hello_decorator(hello)

In [104]:
hello_deco()

Hello there!
stranger...
Have a great time!


In [105]:
hello_deco('John')

Hello there!
Great seeing you John !!
Have a great time!


In [106]:
# Python provides a better syntax, shortcut!
# use @decorter_function name and define the function that needs to be decorated
@hello_decorator
def say_hi():
    """print Hi there! with some decorations."""
    
    print("Hi there!")

In [107]:
say_hi()

Hi there!
stranger...
Have a great time!


In [108]:
# let's check the name name and docstring for say_hi
say_hi.__name__

'wrapper_hello_decorator'

In [109]:
say_hi.__doc__

'Wrapper for hello function.'

In [125]:
# decorated function lost its identify! 
# Let's preserve the identify of functions being decorated...
from functools import wraps

# a simple decorator function to decorate hello
def hello_better(func):
    """Decorate func."""
    
    @wraps(func)
    def wrapper_hello_better(*args, **kwargs):
        """Wrapper for func."""
        # code ...
        # call the actual function
        ret = func(*args, **kwargs)
        # use ret value...
        if not args:
            print("stranger...")
        else:
            print(f"Great seeing you {args[0]}!!")
        print("Have a great time!")
        # code ...
    return wrapper_hello_better

In [126]:
# let's define a new function with hello_better decorator
@hello_better
def greet(name=None):
    """Print Good morning!"""
    
    print('Good morning!')

In [127]:
greet()

Good morning!
stranger...
Have a great time!


In [128]:
greet("Jane")

Good morning!
Great seeing you Jane!!
Have a great time!


In [130]:
greet.__name__

'greet'

In [131]:
greet.__doc__

'Print Good morning!'

## Decorator Pattern

- the Decorator pattern allows us to *wrap* an object that provides core functionalities with other objects
- two primary uses:
 - enhancing the response of a component as it sends data to a second component
 - supporting multiple optional behaviors
     - suitable alternative to multiple inheritance
- create a core object, and then create a decorator wrapping that core
- we can chain the wrapping as the decorator object has the same interface as the core object
- **Core** and all the decorators implement a specific **Interface**
    - the dash lines show "implements" or "realizes"
- when called, the decorator does some added processing before or after calling its wrapped interface
- see how it looks using the UML diagram
![Decorator Pattern UML](resources/decorator_pattern.png)

## Defining function decorator

- we define higher order function that takes function as an argument to be decorated
- we can use wraps function decorator to wrap any function and return it
    - we don't have to but, it preserves the signature of the function being decorated


In [133]:
from functools import wraps
from typing import Callable, Any

def log_args(function: Callable[..., Any]) -> Callable[..., Any]:
    
    @wraps(function)
    def wrapped_function(*args: Any, **kwargs: Any) -> Any:
        print(f"Calling {function.__name__}(*{args}, **{kwargs})")
        #if len(arg) == 3:
        kwargs['1'] = 1
        result = function(*args, **kwargs)
        return result
    
    return wrapped_function

In [134]:
def test1(a: int, b: int, c:int, *args, **kwargs) -> float:
    return sum(range(a, b + 1)) / c

In [135]:
test1 = log_args(test1)

In [136]:
test1(1, 9, 2)

Calling test1(*(1, 9, 2), **{})


22.5

In [137]:
# better option/syntax
@log_args
def test2(a: int, b: int, c:int, *args, **kwargs) -> float:
    return sum(range(a, b + 1)) / c

In [138]:
test2(1, 9, 2)

Calling test2(*(1, 9, 2), **{})


22.5

### Defining class decorator

- in the following example; we have an original class Coffee
- MilkDecorator decorates Coffee by adding cost of milk
- similarly, Coffee can be wrapped with more decorators (e.g., SugarDecorator, WhippedCreamDecorator, etc.)
- this decorator pattern utilizes the **Open-Closed Principle (OCP) of SOLID** principles
    - making it easy to extend functionality without modifying the original class

In [1]:
class Coffee:
    def cost(self):
        return 5
    

#Decorator class
class MilkDecorator:
    def __init__(self, coffee):
        self._coffee = coffee
    
    def cost(self):
        return self._coffee.cost() + 2 # Add cost of mile

In [4]:
# without using the decorator
black_coffee  = Coffee()
print("Black Coffee Cost:", black_coffee.cost())

Black Coffee Cost: 5


In [7]:
# using the decorator
milk_coffee = MilkDecorator(black_coffee)
print("Milk Coffee Cost:", milk_coffee.cost())

Milk Coffee Cost: 7


In [8]:
class SugarDecorator:
    def __init__(self, coffee):
        self._coffee = coffee

    def cost(self):
        return self._coffee.cost() + 1  # Add cost of sugar

In [9]:
# Applying multiple decorators
coffee_with_milk_and_sugar = SugarDecorator(MilkDecorator(Coffee()))
print("Coffee with Milk and Sugar Cost:", coffee_with_milk_and_sugar.cost())

Coffee with Milk and Sugar Cost: 8


In [29]:
# Useful class decorators example
# Generates demo.log when applied to a function
# __call__ overrides when the class is called like a function

from functools import wraps
from typing import Callable, Any
import logging
import time

class NamedLogger:
    # configure logger
    logging.basicConfig(filename="demo.log",
                format='{message}', style='{',
                filemode='w')
        
    def __init__(self, logger_name: str) -> None:
        self.logger = logging.getLogger(logger_name)
        self.logger.setLevel(logging.DEBUG)
        
    def __call__(
           self,
           function: Callable[..., Any]
    ) -> Callable[..., Any]:
        @wraps(function)
        def wrapped_function(*args: Any, **kwargs: Any) -> Any:
            start = time.perf_counter()
            try:
                result = function(*args, **kwargs)
                μs = (time.perf_counter() - start) * 1_000_000
                self.logger.info(
                    f"{function.__name__}(*{args}), {μs:.1f}μs")
                return result
            except Exception as ex:
                μs = (time.perf_counter() - start) * 1_000_000
                self.logger.error(
                    f"{ex}, {function.__name__}(*{args}), { μs:.1f}μs")
                raise
        return wrapped_function

In [30]:
@NamedLogger("app_log")
def test4(median: float, sample: float) -> float:
    return sample - median

In [31]:
test4(4, 10)

6

In [32]:
! cat demo.log

test4(*(4, 10)), 1.0μs
